In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import time
import requests
import json
!pip install redshift_connector
import redshift_connector
import multiprocessing as mp
import json
import os

# Get the absolute path to the directory containing mag_functions.py
# This assumes mag_functions.py is in the same directory as your Jupyter notebook.
current_dir = os.path.dirname(os.path.abspath(__file__))

# Construct the full path to the JSON file
file_path = os.path.join(current_dir, 'api_key_inst_tagger.json')

# Now open the file using the absolute path
with open(file_path, 'r') as f:
    api_key = json.load(f)[0]['api-key']from math import ceil

In [40]:






import mag_functions as F # Moved import statement here

FileNotFoundError: [Errno 2] No such file or directory: 'api_key_inst_tagger.json'

In [25]:
# input that contains a single affiliation string to parse
with open("/content/drive/MyDrive/openalex-institution-parsing/V2/003_Deploy/test_json_single.json", "rb") as f:
    input_json = json.load(f)

In [31]:
# input that contains hard samples to make sure the model does not return errors
with open("/content/drive/MyDrive/openalex-institution-parsing/V2/003_Deploy/test_json_batch_tough.json", "rb") as f:
    input_json = json.load(f)

In [32]:
# input that contains 6 affiliation strings to parse
with open("/content/drive/MyDrive/openalex-institution-parsing/V2/003_Deploy/test_json_batch.json", "rb") as f:
    input_json = json.load(f)

In [33]:
%%time
# testing the call to the API one time
len(json.loads(F.get_tags(input_json, 1)[1]))

NameError: name 'F' is not defined

In [34]:
%%time
# testing the API call 100 times sequentially
for i in range(100):
    F.get_tags(input_json, 1)[1]

NameError: name 'F' is not defined

### Testing throughput with threading

In [35]:
# iteratate through different batch sizes to see which one gives the optimal performance while
# using threading to take advantage of SageMaker parallelism
for i in [20,30]:
    test_payload = input_json*i
    start_time = time.time()
    final_output = []
    with mp.Pool(processes=64) as p:
        results = [p.apply(F.get_tags, args=(test_payload, 1)) for x in range(100)]
    print(f"batch={i}____{(time.time() - start_time) / (i*100*len(input_json))} seconds")

NameError: name 'F' is not defined

### Testing API with data random queried OpenAlex data

In [36]:
# Redshift credentials for querying the OpenAlex database
with open("redshift_creds.txt", "r") as f:
    host = f.readline()[:-1]
    password= f.readline()[:-1]

FileNotFoundError: [Errno 2] No such file or directory: 'redshift_creds.txt'

In [37]:
# Creating a connection
conn = redshift_connector.connect(
     host=host,
     database='dev',
     user='app_user',
     password=password
  )

cursor = conn.cursor()

NameError: name 'host' is not defined

In [ ]:
# Making calls to the API with data that is queried directly from the OpenAlex database
all_input = []
all_output = []
for i in range(25):
    query = f"""select original_affiliation
           from mid.affiliation
           where original_affiliation is not null
           order by RANDOM()
           limit 50"""
    cursor.execute("ROLLBACK;")
    cursor.execute(query)
    test_strings = cursor.fetch_dataframe()
    test_strings.columns = ['affiliation_string']
    test_input = json.loads(test_strings.to_json(orient='records'))
    all_input += test_input
    test_output = json.loads(F.get_tags(test_input, 1)[1])
    print(len(test_output))
    all_output += test_output

In [38]:
# Checking to make sure there are no bad or empty predictions
for i, j in zip(all_input,all_output):
    if j['affiliation_id']:
        print(f"{j['affiliation_id']}")

NameError: name 'all_input' is not defined